In [3]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from tqdm import tqdm
# from sklearn.model_selection import train_test_split
# import os
# import random
# import time

# # Inside the /Multimodal-Deep-Regression/notebooks
import sys
sys.path.append("..")

# from util.utilities import train, evaluate, get_device
# from util.data_utilities import  process_data

# import all the models from models module
# from models import Swin_Transformer
from util.video_swin_transformer import SwinTransformer3D

# device = get_device()

ModuleNotFoundError: No module named 'mmaction'

In [ ]:
process_data(
    input_type='video_pack_1000', 
    addition_parameters={'first_n_videos': 5}, 
    verbose=True,
    device=device,
    skip_frames=True,
    frames_to_skip=2
)

In [ ]:
train_loader, val_loader = get_train_and_val_loader(
    input_type='video_packs_1000',
    batch_size=1,
    verbose=True
)

In [ ]:
Swin_Transformer.SwinTransformer3D(
    
)